In [15]:
import pandas as pd
import csv 

In [17]:
df= pd.read_csv("Suicide_Detection (2).csv", on_bad_lines='skip', encoding = "ISO-8859-1",  quoting=csv.QUOTE_NONE,  lineterminator='\n')

In [18]:
df

,,Unnamed: 0,text,class
2,"""Ex Wife Threatening SuicideRecently I left my wife for good because she has cheated on me twice and lied to me so much that I have decided to refuse to go back to her. As of a few days ago",she began threatening suicide. I have tireles...,but what happens if she really does? What do ...,suicide
3,Am I weird I don't get affected by compliments if it's coming from someone I know irl but I feel really good when internet strangers do it,non-suicide,NaN,NaN
4,"""Finally 2020 is almost over... So I can never hear """"2020 has been a bad year"""" ever again. I swear to fucking God it's so annoying""",non-suicide,NaN,NaN
8,i need helpjust help me im crying so hard,suicide,NaN,NaN
I was later transferred to a fucken group therapy for some reason which made me feel more anxious. Eventually before my last session with a 1 on 1 therapy she showed me my results from a daily check up on my feelings(which was a 2 - step survey for me and my mom/dad ),NaN,NaN,NaN,NaN
...,...,...,...,...
"4. Both start with """"Ga""""",NaN,NaN,NaN,NaN
"1 like = 1 prayer for the Gaymers to rise up.""",non-suicide,NaN,NaN,NaN
310805,"""Why would you should watch Boku No Pico Boku no Pico is a god damn masterpiece.",NaN,NaN,NaN
And if anyone doesnât know,Boku No Pico is about a Alchemist who finds a kid named Pico. He then trains Pico in the ways of Alchemy. Pico now fights with his friends Coco (a dude who wields a fucking Bone sword,itâs badass) and Chico (A man who is trying...,"and pay for her medical bills).""",non-suicide


In [ ]:
def combine_paragraphs(df, column_name):
    """
    Combines paragraphs with the same number into a single text, with each text in a separate row of a new dataframe.
    The input dataframe must have at least two columns, with the first column containing the numbers
    and the second column containing the text.
    """
    current_num = None
    current_text = []
    texts = []
    for num, text in zip(df.iloc[:,0], df[text]):
        if num != current_num:
            if current_num is not None:
                texts.append([' '.join(current_text)])
            current_num = num
            current_text = []
        current_text.append(text)
    texts.append([' '.join(current_text)])
    return pd.DataFrame(texts, columns=[text])

In [ ]:
ls

pytorch_model.bin  sample_data/  trainc.csv


In [ ]:
import pickle
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def preprocess_tweet(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    lowercase_text = re.sub('[\W]+', ' ', text.lower())
    text = lowercase_text+' '.join(emoticons).replace('-', '') 
    return text

In [ ]:
tqdm.pandas()

df['Tweet Text'] = df['Tweet Text'].progress_apply(preprocess_tweet)

100%|██████████| 425/425 [00:00<00:00, 27003.05it/s]


In [ ]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
[w for w in tokenizer_porter('a runner likes running and runs a lot') if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [ ]:
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\(|D|P)',text.lower())
    text = re.sub('[\W]+', ' ', text.lower())
    text += ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in tokenizer_porter(text) if w not in stop]
    return tokenized

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, 
                         preprocessor=None,tokenizer=tokenizer)

In [ ]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='log', random_state=1)

In [ ]:
X = df["text"].to_list()
y = df['class']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,
                                                 y,
                                                 test_size=0.20,
                                                 random_state=0)

In [ ]:
X_train = vect.transform(X_train)
X_test = vect.transform(X_test)

In [ ]:
classes = np.array([0, 1])
clf.partial_fit(X_train, y_train,classes=classes)

In [ ]:
print('Accuracy: %.3f' % clf.score(X_test, y_test))

In [ ]:
clf = clf.partial_fit(X_test, y_test)

In [ ]:
label = {0:'negative', 1:'positive'}
example = ["I'll kill myself am tired of living depressed and alone"]
X = vect.transform(example)
print('Prediction: %s\nProbability: %.2f%%'
      %(label[clf.predict(X)[0]],np.max(clf.predict_proba(X))*100))

In [19]:

import tweepy
import pandas as pd
import pre_trained_model # your pre-trained classification model
from calmharm import CalmHarm # or any other similar intervention tool

# Set up Twitter API credentials
consumer_key = 'your_consumer_key'
consumer_secret = 'your_consumer_secret'
access_token = 'your_access_token'
access_token_secret = 'your_access_token_secret'

# Authenticate your application
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Define your keyword(s) to search for tweets
keywords = ['suicide','void','anger','sad','sadness','my mother died',"I'm going to die","no goals","no purpose","hurt","left out" ,"alone","depression","depressed"]

# Use the tweepy.Cursor() method to search for tweets containing your keyword(s)
tweets = tweepy.Cursor(api.search_tweets,
              q=keywords,
              lang='en',
              tweet_mode='extended').items(50)

# Extract the text of the tweets and store them in a pandas DataFrame
data = pd.DataFrame(data=[tweet.full_text for tweet in tweets], columns=['text'])

# Preprocess the tweet text
# (remove unwanted characters/symbols, tokenize, and convert to numerical representation)
# Assume your pre-trained model takes in a list of tokenized texts
preprocessed_texts = preprocess_text(data['text'])
numerical_representation = convert_to_numerical(preprocessed_texts)

# Use your pre-trained model to classify the tweets as suicide-related or not
predictions = pre_trained_model.predict(numerical_representation)

# Respond to the classified tweets using Calm Harm or a similar intervention tool
for i, prediction in enumerate(predictions):
    if prediction == 1:
        response = CalmHarm().get_tip()
        api.update_status('@' + tweets[i].user.screen_name + ' ' + response, tweets[i].id)

AttributeError: ignored